Connect to Earth Engine 

In [ ]:
import ee
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='sice-geus')


In [37]:
import geemap
import numpy as np
import glob
import os
import pandas as pd
from datetime import datetime, timedelta

In [45]:
############# Setting up EE ##############

### Specify Dates to Process ###

data_folder = r'C:\Users\rabni\OneDrive - GEUS\MODIS\EE\MCD43\BSA' 
re = 'Greenland' 

mo_to_do = ['04','05','06','07','08','09']
sday = '2017-04-01'
eday = '2023-10-02'

dates = pd.date_range(start=sday,end=eday).to_pydatetime().tolist()
dates = [d.strftime("%Y-%m-%d") for d in dates]
dates = [d for d in dates if d.split('-')[1] in mo_to_do]

data_files = glob.glob(data_folder + os.sep + re + os.sep + '*' + os.sep + '*.tif')
data_files_dates = [d.split(os.sep)[-1][:-4] for d in data_files] 
dates_to_do = [d for d in dates if d not in data_files_dates]

print(dates_to_do) 

### Set up Polygon (maybe import) ###

greenland_vertices = [
      [-11.299,81.627],[-11.254,81.161],[-14.744,80.434],[-15.731,80.246],[-16.664,79.564],[-17.426,77.707],
      [-17.094,74.881],[-19.758,73.841],[-21.427,72.621],[-21.445,71.257],[-21.373,69.858],[-25.877,68.350],
      [-32.048,67.543],[-35.924,65.587],[-39.943,64.870],[-40.786,63.022],[-41.916,62.466],[-41.988,61.533],
      [-43.100,59.774],[-44.392,59.541],[-46.132,60.402],[-46.760,60.510],[-47.496,60.600],[-48.384,60.636],
      [-49.093,61.326],[-52.430,64.287],[-52.708,65.175],[-53.462,65.570],[-54.072,66.619],[-53.785,68.575],
      [-55.372,69.911],[-56.090,71.822],[-56.888,73.580],[-58.261,74.800],[-59.723,75.392],[-61.948,75.940],
      [-65.599,75.698],[-69.914,75.859],[-72.605,77.151],[-73.197,78.326],[-72.363,78.730],[-66.855,80.784],
      [-66.308,80.999],[-60.656,81.995],[-58.144,82.237],[-52.089,82.605],[-45.361,83.323],[-40.651,83.520],
      [-35.619,83.879],[-25.392,83.547],[-11.299,81.627],
  ]

# Create a polygon for Greenland using the specified vertices
greenland_polygon = ee.Geometry.Polygon(greenland_vertices)
print('Greenland Polygon:', greenland_polygon.getInfo())




['2020-09-03', '2020-09-04', '2020-09-05', '2020-09-06', '2020-09-07', '2020-09-08', '2020-09-09', '2020-09-10', '2020-09-11', '2020-09-12', '2020-09-13', '2020-09-14', '2020-09-15', '2020-09-16', '2020-09-17', '2020-09-18', '2020-09-19', '2020-09-20', '2020-09-21', '2020-09-22', '2020-09-23', '2020-09-24', '2020-09-25', '2020-09-26', '2020-09-27', '2020-09-28', '2020-09-29', '2020-09-30', '2021-04-01', '2021-04-02', '2021-04-03', '2021-04-04', '2021-04-05', '2021-04-06', '2021-04-07', '2021-04-08', '2021-04-09', '2021-04-10', '2021-04-11', '2021-04-12', '2021-04-13', '2021-04-14', '2021-04-15', '2021-04-16', '2021-04-17', '2021-04-18', '2021-04-19', '2021-04-20', '2021-04-21', '2021-04-22', '2021-04-23', '2021-04-24', '2021-04-25', '2021-04-26', '2021-04-27', '2021-04-28', '2021-04-29', '2021-04-30', '2021-05-01', '2021-05-02', '2021-05-03', '2021-05-04', '2021-05-05', '2021-05-06', '2021-05-07', '2021-05-08', '2021-05-09', '2021-05-10', '2021-05-11', '2021-05-12', '2021-05-13', '2021

In [ ]:


def add_one_day(original_date):
    
    # Convert the input string to a datetime object
    current_date = datetime.strptime(original_date, "%Y-%m-%d")
    # Add one day
    new_date = current_date + timedelta(days=1)
    # Format the result in "YYYY-MM-DD" format
    new_date_formatted = new_date.strftime("%Y-%m-%d")

    return new_date_formatted

for d in dates_to_do:

  date_start = d
  date_end = add_one_day(d)

  year = date_start[:4] 
  output_folder = data_folder+os.sep+re+os.sep+year
    
  # Function to mask clouds and snow
  def mask_clouds_snow(image):
      # Implement your cloud and snow masking logic here
      # Example: mask = image.select('some_band').eq(0)
      # return image.updateMask(mask)
      return image
      
  # Load MODIS albedo data
  albedo = (
      ee.ImageCollection('MODIS/061/MCD43A3')
      .filterBounds(greenland_polygon)
      .filterDate(ee.Date(date_start), ee.Date(date_end))
      .select(['Albedo_BSA_shortwave'])
  )

  # Check the number of images in the collection
  count = albedo.size()
  print('Number of images in the collection:', count.getInfo())

  # Define the projection EPSG:3413
  projection_3413 = "EPSG:3413"

  # Reproject all images to EPSG:3413
  albedo_reprojected = albedo.map(lambda image: image.reproject(crs=projection_3413, scale=500))

  # Create a map with EPSG:3413 projection
  Map = geemap.Map(center=[70, -40], zoom=5)
  Map.add_basemap('SATELLITE')

  # Display the first reprojected image from the collection using geemap
  band_name = 'Albedo_BSA_shortwave'

  first_image_reprojected = albedo.first()
  Map.addLayer(first_image_reprojected.clip(greenland_polygon).select(band_name), {'min': 0, 'max': 32766}, 'Albedo ' + band_name)
  # Add the Greenland polygon to the map
  Map.addLayer(greenland_polygon, {'color': 'FF0000'}, 'Greenland Polygon')
  dates = list(albedo_reprojected.aggregate_array('system:index').getInfo())

  # Export all reprojected images as GeoTIFF to local machine

 
  geemap.ee_export_image_collection(albedo_reprojected, out_dir=output_folder, region=greenland_polygon, scale=500, crs=projection_3413)
  # Display the map in the notebook

  #Map
 

Number of images in the collection: 1
Total number of images: 1

Exporting 1/1: C:\Users\rabni\OneDrive - GEUS\MODIS\EE\MCD43\BSA\Greenland\2020\2020_09_03.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\rabni\OneDrive - GEUS\MODIS\EE\MCD43\BSA\Greenland\2020\2020_09_03.tif


Number of images in the collection: 1
Total number of images: 1

Exporting 1/1: C:\Users\rabni\OneDrive - GEUS\MODIS\EE\MCD43\BSA\Greenland\2020\2020_09_04.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\rabni\OneDrive - GEUS\MODIS\EE\MCD43\BSA\Greenland\2020\2020_09_04.tif


Number of images in the collection: 1
Total number of images: 1

Exporting 1/1: C:\Users\rabni\OneDrive - GEUS\MODIS\EE\MCD43\BSA\Greenland\2020\2020_09_05.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\rabni\OneDrive - GEUS\MODIS\EE\MCD43\BSA\Greenland\2020\2020_09_05.tif


Number of images in the collection: 1
Total number of images: 1

Exporting 1/1: C:\Users\rabni\OneDrive - GEUS\M